## Multiple object tracking with YOLOv3-based object detection

In [1]:
import numpy as np
import cv2 as cv
from motrackers.detectors import YOLOv3
from motrackers import CentroidTracker, CentroidKF_Tracker, SORT, IOUTracker
from motrackers.utils import draw_tracks
import ipywidgets as widgets

In [2]:
VIDEO_FILE = "./../video_data/cars.mp4"
WEIGHTS_PATH = './../pretrained_models/yolo_weights/yolov3.weights'
CONFIG_FILE_PATH = './../pretrained_models/yolo_weights/yolov3.cfg'
LABELS_PATH = "./../pretrained_models/yolo_weights/coco_names.json"

CONFIDENCE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.2
DRAW_BOUNDING_BOXES = True
USE_GPU = False

In [3]:
chosen_tracker = widgets.Select(
    options=["CentroidTracker", "CentroidKF_Tracker", "SORT", "IOUTracker"],
    value='CentroidTracker',
    rows=5,
    description='MOTracker:',
    disabled=False
)
chosen_tracker

Select(description='MOTracker:', options=('CentroidTracker', 'CentroidKF_Tracker', 'SORT', 'IOUTracker'), valu…

In [4]:
if chosen_tracker.value == 'CentroidTracker':
    tracker = CentroidTracker(max_lost=0, tracker_output_format='mot_challenge')
elif chosen_tracker.value == 'CentroidKF_Tracker':
    tracker = CentroidKF_Tracker(max_lost=0, tracker_output_format='mot_challenge')
elif chosen_tracker.value == 'SORT':
    tracker = SORT(max_lost=3, tracker_output_format='mot_challenge', iou_threshold=0.3)
elif chosen_tracker.value == 'IOUTracker':
    tracker = IOUTracker(max_lost=2, iou_threshold=0.5, min_detection_confidence=0.4, max_detection_confidence=0.7,
                         tracker_output_format='mot_challenge')
else:
    print("Please choose one tracker from the above list.")

In [5]:
model = YOLOv3(
    weights_path=WEIGHTS_PATH,
    configfile_path=CONFIG_FILE_PATH,
    labels_path=LABELS_PATH,
    confidence_threshold=CONFIDENCE_THRESHOLD,
    nms_threshold=NMS_THRESHOLD,
    draw_bboxes=DRAW_BOUNDING_BOXES,
    use_gpu=USE_GPU
)

In [10]:
def main(video_path, model, tracker):

    cap = cv.VideoCapture(0)
    while True:
        ok, image = cap.read()

        if not ok:
            print("Cannot read the video feed.")
            break

        image = cv.resize(image, (700, 500))

        bboxes, confidences, class_ids = model.detect(image)
        print(bboxes)
        
        tracks = tracker.update(bboxes, confidences, class_ids)
        
        updated_image = model.draw_bboxes(image.copy(), bboxes, confidences, class_ids)

        updated_image = draw_tracks(updated_image, tracks)

        cv.imshow("image", updated_image)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

In [12]:
main(VIDEO_FILE, model, tracker)

[[ -2  18 267 461]]
[[  0  13 269 468]]
[[  4  15 268 463]]
[[  5  13 268 467]]
[[  7   9 269 475]]
[[ -1  -4 246 494]
 [248 315 432 185]]
[[  9   4 211 489]
 [254 315 426 185]]
[[ -5   8 322 482]]
[[  0   6 262 482]]
[[ -5  10 368 472]]
[[-10  15 378 468]]
[[ -2  18 304 457]]
[[ -3  12 352 471]
 [249 318 433 180]]
[[ -6   5 386 470]
 [244 320 439 176]]
[[ -3  -3 399 492]
 [246 317 435 181]]
[[ -7  -7 400 505]
 [246 316 439 182]]
[[ -9  -4 402 495]
 [244 317 437 181]]
[[ -8  -5 397 500]
 [249 317 432 181]]
[[ -5  -4 397 500]
 [245 318 435 180]]
[[ -5  -6 399 504]
 [246 316 435 183]]
[[ -4  -7 402 506]
 [244 316 439 183]]
[[-10  -8 399 502]
 [247 317 436 181]]
[[ -6  -1 369 492]
 [248 319 435 180]]
[[  4  15 285 459]]
[[  3   1 232 493]
 [249 317 430 183]]
[[  0   4 129 483]
 [249 315 436 185]]
[[  0  11  66 490]]
[[  0  35  62 485]
 [244 314 443 186]]
[[  0   0  71 515]]
[[  0  57  90 405]]
[[  0  93  74 410]]
[[  0  87  70 421]]
[[  1  89  69 418]]
[[  0  78  72 402]]
[[  0  78  75 40

AttributeError: 'tuple' object has no attribute 'flatten'